<a href="https://colab.research.google.com/github/Maximilianwte/GenAI-Course/blob/main/BDD_GenAI_Multimodal_Models_with_Phi_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flash-attn --no-build-isolation --q
from PIL import Image
import pandas as pd
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

model_id = "microsoft/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
def generate_response(image):
  messages = [
    {"role": "user", "content": "<|image_1|>\n What is displayed on the image?"}
]
  prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")
  generation_args = {
      "max_new_tokens": 200,
      "temperature": 0.5
  }
  generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

  generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
  response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  return response